## COLAB NOTEBOOK FOR THE FLAN-T5 SENTENCES TRANSLATIONS OF THE DCEP DE-EN CORPUS

In [ ]:
!pip install torch transformers
!pip install azure-storage-blob
!pip install sentencepiece
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
import requests
import uuid
import json

import torch
import sentencepiece  
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Test Transformers Library
#tokenizer = transformers.BertTokenizer.from_pretrained('google/bert-base-uncased')
#print("Transformers test tokenizer:\n", tokenizer)

# Downloading FLAN-T5-base
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

# Adjust the max_length parameter for model.generate()
max_length = 512  # Increase the value to extend the token limit so the translations don't run short of tokens


print("Model ready")

# Ignore this comments if you are not using any containers

#blob_container_name = "translation"
#input_document_name = "DE-EN-alligned.txt"
#output_document_name = "DE-EN-pred-groundtruth_flant5.txt"

# Translate and write to output file
translated_lines = []

# Ignore this comments if you are not using any containers
#with input_blob_client.download_blob() as blob:
#content = blob.readall().decode('utf-8')
#lines = content.split('\n')

# Read input file
with open('DE-EN-alligned.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Main loop
for line in lines:
    string_1, string_2 = line.strip().split('\t')                        # Our source sentences and references come in format <src \t ref>

    # FLAN-T5 TRANSLATION

    input_text = "translate German to English: " + string_1              # This is the keyword for T5 to translate the text 

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

    outputs = model.generate(input_ids, max_length=max_length)
    translated_string =tokenizer.decode(outputs[0])
    #translated_string = translated_string.replace("<pad>", "").replace("</s>", "").strip()

    # translated string is the string_1 translation

    translated_lines.append(translated_string + '\t' + string_2)         # We append the hypothesis sentences in the same format along with the references for posterior evaluations

    print("Translation number", i, "\n", translated_string + ' -- ' + string_2)
    i += 1

# Save translated output to a file
output_file_path = 'DE-EN-pred-groundtruth_FLANT5.txt'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write('\n'.join(translated_lines))

print("Translation completed.")

In [4]:
# Save translated output to a file
output_file_path = 'DE-EN-pred-groundtruth_FLANT5_77505_v.txt'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write('\n'.join(translated_lines))

print("Translation completed.")

Translation completed.
